In [1]:
import pandas as pd
import numpy as np

X_train = pd.read_csv('./cvss_2022_2024_X_train.csv')
y_train = pd.read_csv('./cvss_2022_2024_y_train.csv')

X_test = pd.read_csv('./cvss_2022_2024_X_test.csv')
y_test = pd.read_csv('./cvss_2022_2024_y_test.csv')

print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

print(y_train['cvssv3_user_interaction'].value_counts(dropna=False) / y_train.shape[0])
print(y_test['cvssv3_user_interaction'].value_counts(dropna=False) / y_test.shape[0])
print(X_train.info())
print(y_train.info())

(64480, 1)
(16120, 1)
(64480, 1)
(16120, 1)
cvssv3_user_interaction
REQUIRED    0.500062
NONE        0.499938
Name: count, dtype: float64
cvssv3_user_interaction
NONE        0.500248
REQUIRED    0.499752
Name: count, dtype: float64
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 64480 entries, 0 to 64479
Data columns (total 1 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   english_description  64480 non-null  object
dtypes: object(1)
memory usage: 503.9+ KB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 64480 entries, 0 to 64479
Data columns (total 1 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   cvssv3_user_interaction  64480 non-null  object
dtypes: object(1)
memory usage: 503.9+ KB
None


In [2]:
label_column_name = "cvssv3_user_interaction"
train_labels = y_train.loc[:, label_column_name]
test_labels = y_test.loc[:, label_column_name]


from sklearn.preprocessing import LabelEncoder
import pickle

le = LabelEncoder()
le.fit(train_labels)
print(le.classes_)

with open("../labels/user_interaction_label.txt", "wb") as f:
    pickle.dump(le.classes_, f)

NUM_CLASSES = len(le.classes_)
print(NUM_CLASSES)

encoded_train_labels = le.transform(train_labels)
encoded_test_labels = le.transform(test_labels)

print(train_labels[:10], encoded_train_labels[:10])
print(len(X_train), len(train_labels), len(X_test), len(test_labels))

['NONE' 'REQUIRED']
2
0        NONE
1        NONE
2        NONE
3        NONE
4    REQUIRED
5        NONE
6    REQUIRED
7        NONE
8    REQUIRED
9        NONE
Name: cvssv3_user_interaction, dtype: object [0 0 0 0 1 0 1 0 1 0]
64480 64480 16120 16120


In [3]:
from transformers import BertTokenizerFast

tokenizer = BertTokenizerFast.from_pretrained('prajjwal1/bert-small')

d:\my-proj\ML - Proje\CVSS-Prediction-BERT_small\venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
train_encodings = tokenizer(X_train.loc[:,"english_description"].tolist(), truncation=True, padding=True, max_length=128)
test_encodings = tokenizer(X_test.loc[:,"english_description"].tolist(), truncation=True, padding=True, max_length=128)

In [5]:
import torch

class CVEDataset(torch.utils.data.Dataset):
    def __init__(self, X, encodings, labels, encoded_labels):
        self.texts = X.loc[:,"english_description"].tolist()
        self.encodings = encodings
        self.labels = labels.tolist()
        self.encoded_labels = encoded_labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['text_labels'] = self.labels[idx]
        item['encoded_labels'] = torch.tensor(self.encoded_labels[idx])
        item['vulnerability_description'] = self.texts[idx]
        
        return item

    def __len__(self):
        return len(self.labels)


In [6]:
train_dataset = CVEDataset(X_train, train_encodings, train_labels, encoded_train_labels)
test_dataset = CVEDataset(X_test, test_encodings, test_labels, encoded_test_labels)

In [7]:
train_dataset[0]

{'input_ids': tensor([  101,  1000,  1037, 18130,  1999,  1037,  3563,  2717, 17928,  2203,
          8400,  1997, 26408,  1050, 20952,  2278,  2071,  3499,  2019, 14469,
          4383,  1010,  2659,  1011, 21598,  1010,  6556, 17346,  2000,  2039,
         11066,  2030,  3972, 12870,  6764,  2006,  2019,  5360,  5080,  1012,
          2023, 18130,  6526,  2138,  1997,  4394, 20104,  7711,  2006,  1996,
          5360,  2717, 17928,  2203,  8400,  1012,  2019, 17346,  2071, 18077,
          2023, 18130,  2011,  6016, 19275, 17928, 11186,  2000,  1996,  5360,
          2203,  8400,  1012,  1037,  3144, 18077,  2071,  3499,  1996, 17346,
          2000,  2039, 11066,  6764,  2046,  1037,  3563, 11661,  2030,  3972,
         12870,  6764,  2013,  1037,  3563, 19622,  2306,  2008, 11661,  1012,
          2023, 18130,  2069, 13531,  1037,  3563,  2717, 17928,  2203,  8400,
          1998,  2515,  2025,  7461,  1996,  4773,  1011,  2241,  2968,  8278,
          1012,  1000,   102,     0,   

In [8]:
from transformers import BertForSequenceClassification

model = BertForSequenceClassification.from_pretrained('prajjwal1/bert-small', num_labels=NUM_CLASSES)
model

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at prajjwal1/bert-small and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 512, padding_idx=0)
      (position_embeddings): Embedding(512, 512)
      (token_type_embeddings): Embedding(2, 512)
      (LayerNorm): LayerNorm((512,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-3): 4 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=512, out_features=512, bias=True)
              (key): Linear(in_features=512, out_features=512, bias=True)
              (value): Linear(in_features=512, out_features=512, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=512, out_features=512, bias=True)
              (LayerNorm): LayerNorm((512,), eps=1e-1

In [9]:
for param in model.base_model.parameters():
    param.requires_grad = False
    
for name, param in model.named_parameters():
    if param.requires_grad:
        print(name, param.data)

classifier.weight tensor([[-0.0154, -0.0156, -0.0241,  ...,  0.0187,  0.0140,  0.0049],
        [-0.0331, -0.0036, -0.0097,  ...,  0.0022, -0.0153, -0.0055]])
classifier.bias tensor([0., 0.])


In [10]:
from torch.utils.data import DataLoader
from transformers import AdamW
import torch.nn.functional as F

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

model.to(device)
model.train()

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)

optim = AdamW(model.parameters(), lr=5e-5)

training_loss_batch = []
training_loss_epoch = []

for epoch in range(3):
    model.train()
    training_loss = 0
    num_correct = 0 
    num_examples = 0
    for batch in train_loader:
        optim.zero_grad()
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['encoded_labels'].to(device)
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs[0]
        loss.backward()
        optim.step()
        training_loss_batch.append(loss.data.item())
        training_loss += loss.data.item() * input_ids.size(0)
        correct = torch.eq(torch.max(F.softmax(outputs.logits, dim=1), dim=1)[1], labels)
        num_correct += torch.sum(correct).item()
        num_examples += correct.shape[0]
    training_loss /= len(train_loader.dataset)
    training_loss_epoch.append(training_loss)
    
    print('Epoch: {}, Training Loss: {}, Training Accuracy = {}'.format(epoch, training_loss, num_correct / num_examples))

  

d:\my-proj\ML - Proje\CVSS-Prediction-BERT_small\venv\lib\site-packages\transformers\optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch: 0, Training Loss: 0.6187873020953043, Training Accuracy = 0.6693548387096774
Epoch: 1, Training Loss: 0.553558181289112, Training Accuracy = 0.7322425558312655
Epoch: 2, Training Loss: 0.5318519908426418, Training Accuracy = 0.7427109181141439


In [11]:
for param in model.base_model.parameters():
    param.requires_grad = True
    
for name, param in model.named_parameters():
    if param.requires_grad:
        # print(name, param.data)
        pass

In [12]:
from torch.utils.data import DataLoader
from transformers import AdamW
import torch.nn.functional as F

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

model.to(device)
model.train()

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)

optim = AdamW(model.parameters(), lr=5e-5)

training_loss_batch = []
training_loss_epoch = []

for epoch in range(3):
    model.train()
    training_loss = 0
    num_correct = 0 
    num_examples = 0
    for batch in train_loader:
        optim.zero_grad()
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['encoded_labels'].to(device)
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs[0]
        loss.backward()
        optim.step()
        training_loss_batch.append(loss.data.item())
        training_loss += loss.data.item() * input_ids.size(0)
        correct = torch.eq(torch.max(F.softmax(outputs.logits, dim=1), dim=1)[1], labels)
        num_correct += torch.sum(correct).item()
        num_examples += correct.shape[0]
    training_loss /= len(train_loader.dataset)
    training_loss_epoch.append(training_loss)
    
    print('Epoch: {}, Training Loss: {}, Training Accuracy = {}'.format(epoch, training_loss, num_correct / num_examples))

  

Epoch: 0, Training Loss: 0.1926736428873803, Training Accuracy = 0.9272177419354839
Epoch: 1, Training Loss: 0.12570519099692598, Training Accuracy = 0.9550868486352357
Epoch: 2, Training Loss: 0.09731902825969928, Training Accuracy = 0.9656017369727047


In [13]:
model.save_pretrained('../models/bert-small-vulnerability_user_interaction-classification')

In [14]:
test_loader = DataLoader(test_dataset, batch_size=16)

model.eval()
num_correct = 0 
num_examples = 0
test_loss = 0
for batch in test_loader:
    input_ids = batch['input_ids'].to(device)
    attention_mask = batch['attention_mask'].to(device)
    labels = batch['encoded_labels'].to(device)
    outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
    loss = outputs[0]
    test_loss += loss.data.item() * input_ids.size(0)
    correct = torch.eq(torch.max(F.softmax(outputs.logits, dim=1), dim=1)[1], labels)
    num_correct += torch.sum(correct).item()
    num_examples += correct.shape[0]
test_loss /= len(test_loader.dataset)

        
print('Test Loss: {}, Test Accuracy = {}'.format(test_loss, num_correct / num_examples))

Test Loss: 0.11094766856187367, Test Accuracy = 0.9637096774193549
